In [1]:
from langgraph.graph import StateGraph
from typing import TypedDict, List, Annotated
from langchain_openai import AzureChatOpenAI
import operator
from langgraph.graph import END
import os
from langgraph.graph import END, StateGraph, START
from langchain_community.document_loaders import PyPDFLoader
from langgraph.graph import END, StateGraph, START
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from typing_extensions import TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureOpenAIEmbeddings

In [2]:
os.environ["OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://expazure-openai.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "0f5cb3c2ad3d40d9a76142c3f49cad9f"

llm = AzureChatOpenAI(
    azure_deployment="gpt35",
    api_version="2024-02-01",
    temperature=0,
    max_tokens=2000,
    timeout=None,
    max_retries=2,
    )

In [3]:
import os
os.environ["OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://expazure-openai.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "0f5cb3c2ad3d40d9a76142c3f49cad9f"

In [4]:



embeddings = AzureOpenAIEmbeddings(
    azure_deployment="embeddingada",
    openai_api_version="2024-02-01",
)

vs_2=FAISS.load_local(f"Vector_data\Install,Upgrade and hotflix StorageGRID", embeddings,allow_dangerous_deserialization=True)
retriever_2 = vs_2.as_retriever()

In [5]:
vs_7=FAISS.load_local(f"Vector_data\Maintain a Storage Grid",embeddings,allow_dangerous_deserialization=True)
retriever_7 = vs_7.as_retriever()

In [6]:

from langchain.tools.retriever import create_retriever_tool
from typing import Literal
retriever_tool_2 = create_retriever_tool(
 
retriever = retriever_2,
name = "Install Upgrade and hotflix StorageGRID",
description ="""
    The document provides detailed instructions on installing, upgrading, and applying hotfixes to 
    StorageGRID, focusing on Red Hat Enterprise Linux. It covers preparation steps, deployment, and 
    configuration of grid nodes, both software-based and appliance nodes. Automation options using orchestration
    frameworks and Python scripts are discussed. It also outlines required information, materials, and hardware
    specifications, emphasizing performance, storage, and network requirements. Additionally, 
    it includes guidelines for downloading and extracting necessary 
    files and managing StorageGRID nodes and resources effectively.
    """
)

retriever_tool_7=create_retriever_tool(
 
retriever = retriever_7,
name = "Maintatin a StorageGRID",
description ="""
    The document details how to maintain a StorageGRID system, including tasks such as decommissioning nodes
    or sites, renaming elements, and performing host and middleware procedures. It outlines prerequisites for 
    maintenance, including understanding the system topology and following instructions precisely. 
    Specific procedures for downloading recovery packages, decommissioning nodes, and considerations for
    each node type are provided, emphasizing the importance of careful planning to ensure data integrity
    and system performance during maintenance activities.
    """
)



In [11]:
question="Does the solution have automatic failure notification via email, text, and other methods?"

topic='Maintain a StorageGRID'

In [12]:
tools = [retriever_tool_2,retriever_tool_7]

In [21]:
llm = AzureChatOpenAI(
azure_deployment="gpt35",
api_version="2024-02-01",
temperature=0,
max_tokens=2000,
timeout=None,
max_retries=2,
)


    
llm_with_tools=llm.bind_tools(tools)

route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant in routing the query to the required vertor database for answering the user question" ),
            ("human", f"This is the question: {question}, this is the topic: {topic}"),
        ]
    )


PROMPT="""This is a fake prompt...

question: {question}

topic: {topic}

Result:"""

question="Does the solution have automatic failure notification via email, text, and other methods?"

topic='Maintain a StorageGRID'


prompt = ChatPromptTemplate.from_template(PROMPT)

chain = (
    {"question": itemgetter("question"), "topic": itemgetter("topic")}
    | route_prompt
    | llm_with_tools 
    | StrOutputParser()
)


response_output = chain.invoke([{"question": question,"topic":topic}])

NameError: name 'itemgetter' is not defined

In [9]:
def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response appended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    
    model = AzureChatOpenAI(
    azure_deployment="gpt35",
    api_version="2024-02-01",
    temperature=0,
    max_tokens=2000,
    timeout=None,
    max_retries=2,
    )
    
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [9]:
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        vectordb: Name of the vectordb
        generation: LLM generation
        documents: list of documents
    """

    question: str
    vectordb: str
    generation: str
    documents: List[str]

In [ ]:
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a 
    user question to the required vectorstore  Use the vectorstore for questions on the required topic. You have to be 
    stringent with the keywords in the question related to these topics. 
    no premable or explaination. Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)

In [ ]:
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})  
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [17]:


def RouteQuery(State):
    
    """Route a user query to the most relevant datasource."""
    
    question=State["question"]
    topic=State["vectordb"]

    datasource: Literal["Install Upgrade and hotflix StorageGRID", "Maintatin a StorageGRID"] = Field(
        ...,
        description="Given a user question and topic choose to route it to the required vectorstore to answer the queries based on the topic.",
    )

    structured_llm_router = llm.with_structured_output(RouteQuery)
    
    llm_with_tools=structured_llm_router.bind_tools(tools)
    
    route_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system),
                ("human", f"This is the question: {question}, this is the topic: {topic}"),
            ]
        )
    
    

    response=structured_llm_router | route_prompt | StrOutputParser()
    
    response_output = question_rewriter.invoke({"question": question,"topic":topic})
    
    return {"generation": response_output}
    
    


In [18]:
workflow=StateGraph(GraphState)
workflow.add_node("Route Query",RouteQuery) 

In [ ]:
workflow.set_entry_point("Route Query")
workflow.add_edge("Analyze Topics",END)